<a href="https://colab.research.google.com/github/chien2734/sgu_data_analyst/blob/anh/01_data_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tiền Xử Lý Dữ Liệu COVID-19

## --- 1. Import thư viện ---

In [169]:
!pip install pycountry

In [170]:
import pandas as pd
import pycountry

## --- 2. Đọc dữ liệu từ WHO ---

In [171]:
# Đọc dữ liệu từ file tải về

url = "https://raw.githubusercontent.com/NAizdabezt/who-covid19-dashboard/main/data/raw/WHO-COVID-19-global-daily-data.csv"
df = pd.read_csv(url)

print("Kích thước dữ liệu:", df.shape)
df.head()

Kích thước dữ liệu: (502800, 8)


,Date_reported,Country_code,Country,WHO_region,New_cases,Cumulative_cases,New_deaths,Cumulative_deaths
0,2020-01-04,AF,Afghanistan,EMR,NaN,0,NaN,0
1,2020-01-04,DZ,Algeria,AFR,NaN,0,NaN,0
2,2020-01-04,AL,Albania,EUR,NaN,0,NaN,0
3,2020-01-04,AI,Anguilla,AMR,NaN,0,NaN,0
4,2020-01-04,AS,American Samoa,WPR,0.0,0,0.0,0


###Kiểm tra và chuẩn hóa tên cột

In [172]:
# Hiển thị các cột ban đầu
print(df.columns)

# Chuẩn hóa tên cột: loại bỏ khoảng trắng và ký tự đặc biệt
df.columns = df.columns.str.strip().str.replace(" ", "_").str.replace("/", "_")
print("\nTên cột sau khi chuẩn hóa:")
print(df.columns)

Index(['Date_reported', 'Country_code', 'Country', 'WHO_region', 'New_cases',
       'Cumulative_cases', 'New_deaths', 'Cumulative_deaths'],
      dtype='object')

Tên cột sau khi chuẩn hóa:
Index(['Date_reported', 'Country_code', 'Country', 'WHO_region', 'New_cases',
       'Cumulative_cases', 'New_deaths', 'Cumulative_deaths'],
      dtype='object')


### Chuyển kiểu dữ liệu ngày tháng

In [173]:
# Chuyển cột Date_reported sang datetime
df['Date_reported'] = pd.to_datetime(df['Date_reported'], errors='coerce')

# Kiểm tra xem có giá trị null sau khi chuyển không
print("Số dòng bị lỗi ngày:", df['Date_reported'].isna().sum())

Số dòng bị lỗi ngày: 0


###Kiểm tra dữ liệu thiếu và trùng

In [174]:
# Số lượng giá trị null
print("\nSố lượng giá trị thiếu theo cột:")
print(df.isnull().sum())

# Kiểm tra dòng trùng
print("\nSố dòng trùng:", df.duplicated().sum())

# Loại bỏ dòng trùng (nếu có)
df = df.drop_duplicates()


Số lượng giá trị thiếu theo cột:
Date_reported             0
Country_code           2095
Country                   0
WHO_region                0
New_cases            291580
Cumulative_cases          0
New_deaths           348527
Cumulative_deaths         0
dtype: int64

Số dòng trùng: 0


###Kiểm tra và xử lý dữ liệu bất thường

In [175]:
# Xem các giá trị âm trong số ca hoặc tử vong
cols_check = ['New_cases', 'Cumulative_cases', 'New_deaths', 'Cumulative_deaths']
for col in cols_check:
    neg = df[df[col] < 0]
    print(f"{col}: {len(neg)} dòng giá trị âm")

# Nếu có giá trị âm, thay bằng NaN (coi là dữ liệu lỗi)
df[cols_check] = df[cols_check].apply(lambda x: x.mask(x < 0))

New_cases: 60 dòng giá trị âm
Cumulative_cases: 0 dòng giá trị âm
New_deaths: 30 dòng giá trị âm
Cumulative_deaths: 0 dòng giá trị âm


### Tổng quan nhanh dữ liệu sau khi làm sạch

In [176]:
print("\nSau khi làm sạch:")
print(df.info())
print(df.describe())


Sau khi làm sạch:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 502800 entries, 0 to 502799
Data columns (total 8 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Date_reported      502800 non-null  datetime64[ns]
 1   Country_code       500705 non-null  object        
 2   Country            502800 non-null  object        
 3   WHO_region         502800 non-null  object        
 4   New_cases          211160 non-null  float64       
 5   Cumulative_cases   502800 non-null  int64         
 6   New_deaths         154243 non-null  float64       
 7   Cumulative_deaths  502800 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 30.7+ MB
None
                       Date_reported     New_cases  Cumulative_cases  \
count                         502800  2.111600e+05      5.028000e+05   
mean   2022-11-16 00:00:00.000000256  3.688310e+03      2.076666e+06   
min     

## --- 3. Đọc dữ liệu dân số từ WB  ---

#### 1. Hàm chuyển Alpha-2 sang Alpha-3

In [177]:
def alpha2_to_alpha3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None

#### 2. Hàm dò tên quốc gia nếu mã lỗi

In [178]:
def get_country_code3_by_name(name):
    try:
        choices = [c.name for c in pycountry.countries]
        match = process.extractOne(name, choices)
        if match and match[1] > 80:
            return pycountry.countries.get(name=match[0]).alpha_3
    except:
        return None

#### 3. Lấy dữ liệu COVID mới nhất theo từng quốc gia

In [179]:
latest = (
    df.sort_values('Date_reported')
      .drop_duplicates('Country', keep='last')
      .reset_index(drop=True)
)


#### 4. Chuẩn hóa tên quốc gia

In [180]:
latest['Country_clean'] = latest['Country'].replace({
    'Cura�ao': 'Curacao',
    'Saint Martin (French part)': 'Saint Martin',
    'Sao Tome and Principe': 'Sao Tome and Principe',
    'British Virgin Islands': 'British Virgin Islands',
    'Turks and Caicos Islands': 'Turks and Caicos Islands',
    'Pitcairn': 'Pitcairn Islands',
    'Montenegro': 'Montenegro',
    'Gambia': 'Gambia',
    'Falkland Islands (Malvinas)': 'Falkland Islands',
    'United States Virgin Islands': 'United States Virgin Islands',
    'Bonaire, Sint Eustatius and Saba': 'Bonaire',
    'Bahamas': 'Bahamas',
    'Andorra': 'Andorra',
    'occupied Palestinian territory, including east Jerusalem': 'Palestine',
    'Sint Maarten (Dutch part)': 'Sint Maarten',
    'R�union': 'Reunion',
    'North Macedonia': 'Macedonia',
    'Saint Barthelemy': 'Saint Barthelemy',
    'Netherlands (Kingdom of the)': 'Netherlands',
    'Micronesia (Federated States of)': 'Micronesia',
    'Bolivia (Plurinational State of)': 'Bolivia',
    'Czechia': 'Czech Republic',
    'Eswatini': 'Swaziland',
    'Cabo Verde': 'Cape Verde',
    'Brunei Darussalam': 'Brunei',
    'Timor-Leste': 'East Timor',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Holy See': 'Vatican City',
    'Kosovo (in accordance with UN Security Council Resolution 1244 (1999))': 'Kosovo',
    'Côte d’Ivoire': 'Ivory Coast',
    'Türkiye': 'Turkey',
    'Viet Nam': 'Vietnam',
    "Lao People's Democratic Republic": 'Laos',
    'Iran (Islamic Republic of)': 'Iran',
    'Republic of Korea': 'South Korea',
    'Syrian Arab Republic': 'Syria',
    'Russian Federation': 'Russia',
    'Venezuela (Bolivarian Republic of)': 'Venezuela'
})

#### 5. Lấy mã ISO3

In [181]:
latest['Country_code3'] = latest['Country_code'].apply(alpha2_to_alpha3)

latest['Country_code3'] = latest.apply(
    lambda r: get_country_code3_by_name(r['Country_clean']) if pd.isna(r['Country_code3']) else r['Country_code3'],
    axis=1
)

special_codes = {
    'Taiwan': 'TWN',
    'Palestine': 'PSE',
    'Kosovo': 'XKX',
    'Namibia': 'NAM'
}

latest['Country_code3'] = latest.apply(
    lambda r: special_codes.get(r['Country_clean'], r['Country_code3']),
    axis=1
)


####6. Lấy dữ liệu dân số mới nhất từ GitHub (World Bank mirror)

In [182]:
url = "https://raw.githubusercontent.com/datasets/population/master/data/population.csv"
pop = pd.read_csv(url)
pop_latest = (
    pop.sort_values('Year')
       .drop_duplicates('Country Code', keep='last')
       .rename(columns={'Country Code': 'Country_code3', 'Value': 'Population'})
       [['Country_code3', 'Population']]
)

####7. Thêm dân số thủ công cho vùng/lãnh thổ bị thiếu

In [183]:
manual_pop = {
    # bổ sung 11 vùng thiếu
    'PCN': 50,
    'MSR': 4900,
    'BLM': 10000,
    'GUF': 300000,
    'MTQ': 370000,
    'BES': 26000,
    'VAT': 800,
    'GLP': 400000,
    'REU': 870000,
    'SHN': 6000,   # Saint Helena
    'MYT': 280000, # Mayotte
    'TKL': 1500,   # Tokelau
    'WLF': 12000,  # Wallis and Futuna
    'NIU': 1600,   # Niue
    'SPM': 6000,   # Saint Pierre and Miquelon
    'COK': 18000,  # Cook Islands
    'AIA': 15000,  # Anguilla
    'GGY': 67000,  # Guernsey
    'JEY': 108000, # Jersey
    'FLK': 3500,   # Falkland Islands

}
manual_df = pd.DataFrame(list(manual_pop.items()), columns=['Country_code3', 'Population'])
pop_latest = pd.concat([pop_latest, manual_df], ignore_index=True)

####8. Ghép dân số vào bảng latest

In [184]:
latest = latest.merge(pop_latest, on='Country_code3', how='left')

####9. Loại bỏ dòng không phải quốc gia

In [185]:
latest = latest[~latest['Country'].str.contains("International conveyance|vessel", case=False, na=False)]

####10. Tính tỷ lệ

In [186]:
latest['Cases_per_million'] = latest.apply(
    lambda r: r['Cumulative_cases'] / (r['Population'] / 1_000_000)
    if pd.notnull(r['Population']) and r['Population'] > 0 else None,
    axis=1
)
latest['Fatality_rate'] = latest.apply(
    lambda r: r['Cumulative_deaths'] / r['Cumulative_cases'] * 100
    if r['Cumulative_cases'] > 0 else 0,
    axis=1
)

####11. Kiểm tra

In [187]:
print("⚠️ Quốc gia thiếu mã ISO3:", latest['Country_code3'].isna().sum())
print("⚠️ Quốc gia thiếu dân số:", latest['Population'].isna().sum())
print("Danh sách thiếu dân số:", latest[latest['Population'].isna()]['Country_clean'].tolist())

⚠️ Quốc gia thiếu mã ISO3: 0
⚠️ Quốc gia thiếu dân số: 0
Danh sách thiếu dân số: []
